In [197]:
import random, copy
import cv2
import numpy as np
import random

In [198]:
def objective_function(position):
    # Create binary mask based on threshold value
    threshold_value = position
    _, mask = cv2.threshold(img, threshold_value, 255, cv2.THRESH_BINARY)

    # Compute percentage of foreground pixels in mask
    n_pixels = mask.flatten().shape[0]
    n_foreground_pixels = np.count_nonzero(mask == 255)
    if n_pixels > 0:
        percent_foreground_pixels = n_foreground_pixels / n_pixels
    else:
        percent_foreground_pixels = 0

    # Return negative of percentage of foreground pixels as fitness value
    return percent_foreground_pixels

In [199]:
### POPULATION LEVEL ###
def gpso_search():
    population = create_pop()
    personal_best_population = copy.copy(population)
    global_best, global_best_fitness = get_best_pop(personal_best_population)
    #print(global_best_fitness)
    for _ in range(NUM_GENERATIONS):
       # print(population,'\n')
        population = convex_combination_pop(population, personal_best_population, [global_best]*POPULATION_SIZE)
        population = mutation_pop(population)
        personal_best_population = selection_pop(personal_best_population, population)
        global_best, global_best_fitness = get_best_pop(personal_best_population)
        #print(global_best_fitness)
    return (global_best, global_best_fitness) 

In [200]:
### INDIVIDUAL LEVEL ###

def create_pop():
    return [ create_ind() for _ in range(POPULATION_SIZE) ]

def get_best_pop(population):
    best_ind = max(population, key=evaluate_ind)
   # print(best_ind)
    a=[objective_function(best_inds) for best_inds in best_ind]
    return a,evaluate_ind(a)

def convex_combination_pop(pop1, pop2, pop3):
    return [ convex_combination_ind([ind1, ind2, ind3]) for (ind1, ind2, ind3) in zip(pop1, pop2, pop3) ]

def mutation_pop(population):
    return [ mutation_ind(individual) for individual in population ]

def selection_pop(pop1, pop2):
    return [ max(ind1, ind2, key=evaluate_ind) for (ind1, ind2) in zip(pop1, pop2) ]



In [201]:
### REPRESENTATION LEVEL ###

def create_ind():
    return [ random.randint(0, 255) for _ in range(INDIVIDUAL_SIZE) ]

def evaluate_ind(individual): #one_max
    return sum(individual)

def convex_combination_ind(mating_pool):
    transposed_mating_pool=zip(*mating_pool)
    return map(random.choice, transposed_mating_pool)

def mutation_ind(individual):
    return [ 1-bit if random.random()<1.0/INDIVIDUAL_SIZE else bit for bit in individual ]


In [229]:
img = cv2.imread('2.jpg',0)

### EXPERIMENTS ###

NUM_GENERATIONS = 50
POPULATION_SIZE = 100
INDIVIDUAL_SIZE = 255
global_best2, global_best_fitness2=gpso_search()

In [230]:
print(global_best_fitness2)

54.200826404635954


In [231]:

threshold_value = int(global_best_fitness2)
_, mask = cv2.threshold(img, threshold_value, 255, cv2.THRESH_BINARY)

# Apply mask to original image to create segmented image
segmented_image = cv2.bitwise_and(img, img, mask=mask)

In [232]:
cv2.imshow("Segmented Image2", segmented_image)
cv2.imshow("mask", mask)
cv2.waitKey(0)
cv2.destroyAllWindows()